<a href="https://colab.research.google.com/github/seongyongju/SAFE_AI_Project_Proposal/blob/main/deep_paper_gestalt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **Deep Paper Gestalt**

Deep Paper Gestalt, arXiv:1812.08775, 2018
https://arxiv.org/pdf/1812.08775.pdf

TL;DR: Try out if your paper passes our deep paper gestalt classifier!

Jia-Bin Huang (jbhuang@vt.edu)


In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torchvision import models, transforms
from torch.autograd import Variable

import matplotlib.pyplot as plt
import cv2

!pip install pdf2image
!apt-get install -y poppler-utils
from pdf2image import convert_from_path
from PIL import Image


Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.62.0-2ubuntu2.10).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [ ]:
def paper_to_image(pdf_file_name: str)-> str:
  """ Convert a paper PDF to an image with a 2 x 4 panel """
  images = convert_from_path(pdf_file_name, 200, None, None, None, 'jpg', 1)
  num_page = len(images)
  if num_page > 6:
      # Create an empty image with a 2 x 4 pages panel
      total_width = 0
      max_height  = 0

      new_width  = round(images[0].width*4)
      new_height = round(images[0].height*2)

      new_im = Image.new('RGB', (new_width, new_height), (255,255,255))

      # Copy and paste pages from pages 1-4
      x_offset = 0
      y_offset = 0
      for i in range(4):
        new_im.paste(images[i], (x_offset, y_offset))
        x_offset += images[i].size[0]

      # Copy and paste pages from pages 5-8
      x_offset = 0
      y_offset += images[i].size[1]

      for i in range(4,8):
        if i < num_page:
          new_im.paste(images[i], (x_offset, y_offset))
          x_offset += images[i].size[0]

  else:
    BasicException('We process PDF with at least 7 pages long.')

  # Save the image as a JPG
  img_file_name = pdf_file_name[0:-4]+'.jpg'
  new_im = new_im.resize((3400, 2200))
  new_im.save(img_file_name)

  # Resize the image to [680, 440] and remove header (to avoid data leakage)
  img = cv2.imread(img_file_name)
  img = cv2.resize(img, dsize=(680,440), interpolation=cv2.INTER_AREA)
  img[0:15, 0:150] = 255 # remove header (to avoid data leakage)
  cv2.imwrite(img_file_name, img)

  print('Converted the PDF ' + pdf_file_name)

  return img_file_name

In [ ]:
# Set up the K-80 GPU from Google Colab: Edit -> Network settings -> Hardware accelerator
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Download pre-trained weight
!wget http://filebox.ece.vt.edu/~jbhuang/project/gestalt/PaperNet.pth

# Load the paper gestalt model
# (pre-trained on CVPR/ICCV 2013-2017 conference/workshop papers)
gestalt_model = models.resnet18(pretrained=False)
gestalt_model.avgpool = nn.AdaptiveAvgPool2d((1, 1))
gestalt_model.fc = nn.Linear(gestalt_model.fc.in_features, 2)
gestalt_model.load_state_dict(torch.load('PaperNet.pth'))
gestalt_model = gestalt_model.to(device)


--2019-11-16 04:07:06--  http://filebox.ece.vt.edu/~jbhuang/project/gestalt/PaperNet.pth
Resolving filebox.ece.vt.edu (filebox.ece.vt.edu)... 128.173.88.43
Connecting to filebox.ece.vt.edu (filebox.ece.vt.edu)|128.173.88.43|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://filebox.ece.vt.edu/~jbhuang/project/gestalt/PaperNet.pth [following]
--2019-11-16 04:07:06--  https://filebox.ece.vt.edu/~jbhuang/project/gestalt/PaperNet.pth
Connecting to filebox.ece.vt.edu (filebox.ece.vt.edu)|128.173.88.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44768048 (43M)
Saving to: ‘PaperNet.pth.1’

PaperNet.pth.1      100%[===================>]  42.69M  11.1MB/s    in 5.3s    

2019-11-16 04:07:12 (7.98 MB/s) - ‘PaperNet.pth.1’ saved [44768048/44768048]



In [ ]:
# Upload your paper PDF in CVPR/ICCV template
from google.colab import files

uploaded = files.upload()

for file_name in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=file_name, length=len(uploaded[file_name])))

Saving CVPR20_Graph_SSL.pdf to CVPR20_Graph_SSL.pdf
User uploaded file "CVPR20_Graph_SSL.pdf" with length 5368395 bytes


In [ ]:
# Now, let's check if your paper pass the paper gestalt test

preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

img_file_name = paper_to_image(file_name)

# Temporature
T = 5

with torch.no_grad():
  im = Image.open(img_file_name)
  im_tensor = preprocess(im)
  im_tensor = im_tensor.to(device)
  im_tensor = im_tensor.unsqueeze(0)

  pred_logit = gestalt_model(im_tensor)
  pred_logit = pred_logit.to(torch.device("cpu")).numpy() / T

  pred_prob = softmax(pred_logit[0])

  print('Our classifier predicts Pr[Good paper|{paper_pdf}] = {prob:.02f}% '\
        .format(paper_pdf= file_name, prob = pred_prob[0]*100))

Converted the PDF CVPR20_Graph_SSL.pdf
Our classifier predicts Pr[Good paper|CVPR20_Graph_SSL.pdf] = 85.25% 
